## Explore a particular set of outputs

For a given region and time that you have already preprocessed, explore it on a map.

In [ ]:
from fireatlas import FireIO, preprocess,  postprocess
import branca.colormap as cm
import folium

tst = [2020, 9, 5, "AM"]
ted = [2020, 9, 8, "PM"]
region = ("Creek", [-119.5, 36.8, -118.9, 37.7])

region = preprocess.read_region(region, location="local")
df = postprocess.read_allpixels(tst, ted, region, location="local")
gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local").reset_index()

In [ ]:
mymap = folium.Map(zoom_start=12)

# put the region on the map
folium.GeoJson(
    region[1],
    style_function=lambda x: {'fillColor': 'pink', 'color': 'red', 'weight': 2}
).add_to(mymap)

for i, t in enumerate(df.t.unique()[::-1]):
    color = cm.linear.viridis(1 - i/len(df.t.unique()))

    # add allpixels
    data = df[df["t"] == t]
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row['Lat'], row['Lon']],
            radius=1,  # Define the circle size
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6
        ).add_to(mymap)

    # add allfires perimeters
    data = gdf[(gdf["t"] == t) & (~gdf["invalid"])]
    folium.GeoJson(
        data["hull"],
        name=f"perimeter at {t}",
        color=color,
        fill_opacity=0.2,
        weight=2, 
    ).add_to(mymap)
    
mymap